In [ ]:
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [ ]:
import pandas as pd
data = pd.read_csv("./../assets/test.csv",header=None)
data = data.values
for i in range(len(data)):
    data[i][0]=i

In [ ]:
all_embs =  []
from tqdm import tqdm
for el in tqdm(data):
    embs = model_emb.encode([el[2]], normalize_embeddings=True)
    all_embs.append((el[0],embs))

In [ ]:
import chromadb
client = chromadb.Client()

collection = client.create_collection("sample_collection7")
abc = [el[1][0].tolist() for el in all_embs]
collection.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in data[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

In [ ]:
def get_documents(q):
    instruction = "Represent this sentence for searching relevant passages:"
    query = instruction + " " + q
    query = model_emb.encode(query, normalize_embeddings=True)
    results = collection.query(query_embeddings=[query.tolist()],n_results=10)
    return results

In [ ]:
def get_answer(question):
    docs = [data[int(el)][2] for el in get_documents(question)["ids"][0]]
    return docs

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM  
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")  
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta").half().cuda()

In [ ]:
# query = "Why did IBM acquired some Danish IT firms?"

In [ ]:
# prompt = f"Answer the question using the provided context. Your answer should be in your own words and be no longer than 30 words. You should ignore documents that do not contain relevant information. if you can't answer the question, say so. Return only the Answer to the Question, if you answered the question, end the sentence, do not ask further questions. \n\n Context: {context} \n\n Question: {query} \n\n Answer:"

In [ ]:
def chat_msg(query, history):
    docs = get_answer(query)
    context = ""
    for i in range(len(docs)):
        context+=f"{i+1}. {docs[i]}  "
    prompt = f"Answer the question using the provided context. Your answer should be in your own words and be no longer than 30 words. You should ignore documents that do not contain relevant information. if you can't answer the question, say so. Return only the Answer to the Question, if you answered the question, end the sentence, DO NOT ASK FURTHER QUESTIONS!. \n\n Context: {context} \n\n Question: {query} \n\n Answer:"
    with torch.inference_mode():
        qqq = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
        a = model.generate(qqq,max_new_tokens=100)
        b = tokenizer.decode(a[0][len(qqq[0]):])
        return b

In [ ]:
get_answer("What is the Pride of Aloha?")

In [ ]:
chat_msg("Why there is a conflict of interest existing between Chelsea and CSKA Moscow?",None)

In [ ]:
import gradio as gr

# def echo(message, history):
#     return message

demo = gr.ChatInterface(fn=chat_msg, examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()